## Data exploration

#### Loading data

In [1]:
import pandas as pd
data_csv = pd.read_csv('./data/data-2015.csv')
data_csv.head()

,user_id,log_id,sequence_id,correct
0,50121,167478035,7014,0.0
1,50121,167478043,7014,1.0
2,50121,167478053,7014,1.0
3,50121,167478069,7014,1.0
4,50964,167478041,7014,1.0


#### Data Description

The data file `data-2015.csv` is the ASSISTments dataset taken from: https://sites.google.com/site/assistmentsdata/datasets/2015-assistments-skill-builder-data. This dataset records the process of students answering questions in problem sets and their performance. The platform being used to collect data is: https://new.assistments.org/

Each row of the dataset contains the information of one student answering one question in a particular problem set.

- `user_id`: The id number assigned to each students. Each student has one unique id.
- `log_id`: The unique log id appeared in the database when the student answer current question. The larger log id is, the later the question was answered.
- `sequence_id`: The id of the problem sets that the question is in.
- `correct`: The correctness of the student's answer. It ranges from 0.0 to 1.0, where 0.0 means that the student answered incorrectly on the first attempt and 1.0 means that the student answered correctly on the first attempt. There may be values in between the range, however, for this current model, we only focus on the correct values of 0 and 1.

#### Data analysis

In [2]:
data_csv.describe()

,user_id,log_id,sequence_id,correct
count,708631.000000,7.086310e+05,708631.000000,708631.000000
mean,296232.978276,1.695323e+08,22683.474821,0.725502
std,48018.650247,3.608096e+06,41593.028018,0.437467
min,50121.000000,1.509145e+08,5898.000000,0.000000
25%,279113.000000,1.660355e+08,7020.000000,0.000000
50%,299168.000000,1.704579e+08,9424.000000,1.000000
75%,335647.000000,1.723789e+08,14442.000000,1.000000
max,362374.000000,1.754827e+08,236309.000000,1.000000


The dataset includes **708631 rows**.

In [3]:
print('Part of missing values for every column')
print(data_csv.isnull().sum() / len(data_csv))

Part of missing values for every column
user_id        0.0
log_id         0.0
sequence_id    0.0
correct        0.0
dtype: float64


As the data includes no missing values, we do not have to handle missing values

In [4]:
print('The number of unique values for every column')
print(data_csv.nunique())

The number of unique values for every column
user_id         19917
log_id         708631
sequence_id       100
correct            11
dtype: int64


The dataset records 19917 students answering 100 problem sets. Each row has one unique `log_id`. Lastly, there are multiple values of `correct`, however we only focus on the value of 0 and 1.

Some of the necessary EDA steps when dealing with tabular data in machine learning are:

- Check the frequency counts and distribution of the target variable to see if there is any class imbalance or skewness.
- Check the distribution of each feature to see if there are any outliers, missing values, or anomalies.
- Check the correlation matrix and scatter plots to see how different features are related to each other and to the target variable.
- Check the statistical significance of the features using hypothesis testing or feature selection methods to see which ones are important for predicting the target variable.
- Check the distributional fit of the features and apply transformations if needed to make them more suitable for machine learning algorithms.
- Check for multicollinearity and remove redundant features if needed to avoid overfitting and improve model performance.

In [9]:
print('Correlation between columns')
print(data_csv.corr())

Correlation between columns
              user_id    log_id  sequence_id   correct
user_id      1.000000  0.515198    -0.040369  0.028428
log_id       0.515198  1.000000    -0.009621  0.054341
sequence_id -0.040369 -0.009621     1.000000 -0.069912
correct      0.028428  0.054341    -0.069912  1.000000


There are little correlation from other columns to the `correct` column

In [11]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

#find design matrix for regression model using 'rating' as response variable
y, X = dmatrices('correct ~ user_id+log_id+sequence_id', data=data_csv, return_type='dataframe')
#create DataFrame to hold VIF values
vif_df = pd.DataFrame()
vif_df['variable'] = X.columns

#calculate VIF for each predictor variable
vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

#view VIF for each predictor variable
print(vif_df)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


      variable       VIF
0    Intercept  0.000000
1      user_id  1.363667
2       log_id  1.361571
3  sequence_id  1.001803


The multicollinearity is not present in this dataset as VIF is smaller than 5

## Data preprocessing